# Set up for model experiments

We'll do the following here:

- Create distinct catalogs
- Document parameters changes that will accompany each
- Create yamls for each experiment

# Catalogs

In [45]:
import pandas as pd
from pathlib import Path
import yaml
import os
import re
import glob 
import geopandas as gpd

In [11]:
catalog = pd.read_csv("../data/ftw-mappingafrica-combined-catalog.csv")
catalog.query("dataset == 'ftw'").to_csv("../data/ftw-catalog.csv", index=False)

## Single Time Point

#### Combined catalogs

Including small test catalogs 

Update: catalog will be adjusted to drop Portugal and presence-only data from validation set for FTW

In [ ]:
catalog = pd.read_csv("../data/ftw-mappingafrica-combined-catalog.csv")
# catalog["split"]

# 22 records in India had no split assigned--placed them in train
catalog["split"] = catalog["split"].replace("none", "train")

catalog['split'] = catalog['split'].astype(str).str.strip().str.lower()
catalog['country'] = catalog['country'].astype(str).str.strip().str.lower()
catalog['null_prop'] = pd.to_numeric(catalog['null_prop'], errors='coerce')  # NaN if non-numeric

mask = ~(
    catalog['split'].isin(['validate', 'test'])
    & (
        (catalog['null_prop'] > 0) | (catalog['country'] == 'portugal')
    )
)
catalogr = catalog[mask].copy().reset_index(drop=True)

# FTW and full catalogs dropping Portugal and presence-only data from val/test
(catalogr.query("dataset == 'ftw'")
 .to_csv("../data/ftw-catalog2.csv", index=False))
(catalogr
 .to_csv("../data/ftw-mappingafrica-combined-catalog2.csv", index=False))

# small FTW for rapid model testing
ftw_small = (
    catalogr
     .query("dataset == 'ftw'")
     .sample(10000, random_state=42)
).reset_index(drop=True)#.to_csv("../data/ftw-catalog-small.csv", index=False))
# ftw_small["split"].value_counts()
ftw_small.to_csv("../data/ftw-catalog-small.csv", index=False)


Make another small catalog for debugging, including null prop > 0 in validation data

In [ ]:
ftw_catalog = pd.read_csv("../data/ftw-catalog.csv")
ftw_small_debug = (ftw_catalog
                   .sample(1000, random_state=42)
                   .reset_index(drop=True))
# ftw_small_debug.query("split == 'validate' & null_prop > 0")
ftw_small_debug.to_csv("../data/ftw-catalog-small-debug.csv", index=False)

#### Mapping Africa

In [ ]:
catalog = pd.read_csv("../data/ftw-mappingafrica-combined-catalog.csv")
(catalog.query("dataset == 'mappingafrica'")
 .to_csv("../data/mappingafrica-catalog.csv", index=False))

Mapping Africa countries with >30 validation samples

In [ ]:
ma_catalog = pd.read_csv("../data/mappingafrica-catalog.csv")
ma_countries = (ma_catalog.query("split == 'validate'")
 .country
 .value_counts()
 .reset_index()
 .query("count > 30")
 .sort_values("country")
 .drop("count", axis=1)
 )
with open("../scripts/ma-countries.txt", "w") as f:
    for c in ma_countries.country.tolist():
        f.write(f"{c}\n")

#### Inference catalogs 

For inference testing, both chips and tiles

In [ ]:

# # Recursively find all files in /Users/lestes/images/tiles and subfolders
tiles_list = [
    os.path.relpath(f, "/Users/lestes/images/tiles")
    for f in glob.glob("/Users/lestes/images/tiles/**/*", recursive=True)
    if os.path.isfile(f) and f.endswith("cog.tif")
    # (os.path.basename(f).startswith(("tile910", "tile513", "tile765"))      
]

# tile numbers of interest
tile_ids = ["910217", "910218", "513706", "513726", "765503"]  

# Filter tiles_list for specified tile_ids and keep only the last (latest year) 
# For each tile_id, keep only the image from the most recent year (folder)
tile_dict = {}
for f in tiles_list:
  basename = os.path.basename(f)
  for tid in tile_ids:
    if basename.startswith(f"tile{tid}"):
      year_folder = os.path.dirname(f)
      try:
        year_int = int(year_folder)
      except ValueError:
        continue  # skip if folder is not a year
      # Keep the file if it's the first seen or has a greater year
      if tid not in tile_dict or year_int > int(os.path.dirname(tile_dict[tid])):
        tile_dict[tid] = f
tiles_selected = list(tile_dict.values())
                 
tile_catalog = pd.DataFrame(
    [{"name": re.sub("tile", "", os.path.basename(f).split("_")[0]),  
      # "year": os.path.dirname(f), 
      "date": f"{os.path.basename(f).split("_")[1]}-15",
      "window_b": f}
      for f in tiles_selected]
).sort_values(by=["name", "date"]).reset_index(drop=True)
# tile_catalog.to_csv("../data/mappingafrica-tile-catalog.csv", index=False)
# (tile_catalog.query("name in ['910217', '910218'] & year == '2024'")
#  .to_csv("../data/mappingafrica-tile-catalog-small.csv", index=False))

tile_catalog


,name,date,window_b
0,513706,2023-11-15,2023/tile513706_2023-11_buf179_cog.tif
1,513726,2023-11-15,2023/tile513726_2023-11_buf179_cog.tif
2,765503,2022-03-15,2022/tile765503_2022-03_buf179_cog.tif
3,910217,2024-06-15,2024/tile910217_2024-06_buf179_cog.tif
4,910218,2024-06-15,2024/tile910218_2024-06_buf179_cog.tif


In [8]:
pth = "~/Dropbox/projects/activelearning/mappingafrica/campaigns/data/grids"
tiles = [gpd.read_file(f"{pth}/{cntry}_tiles.geojson")[["tile", "geometry"]] 
         for cntry in ["ghana", "congo", "zambia"]]
tiles = pd.concat(tiles, axis=0)
tiles["tile"] = tiles["tile"].astype(int).astype(str)
tiles = tiles.query("tile in @tile_ids").reset_index(drop=True)

tiles_gdf = (
    pd.merge(tile_catalog, tiles, left_on="name", right_on="tile", how="inner")
    .drop(columns=["tile"])
    .pipe(gpd.GeoDataFrame, geometry="geometry", crs="EPSG:4326")
)
tiles_gdf.to_file("../data/mappingafrica-tile-catalog-small.geojson", 
                  driver="GeoJSON")


#### FTW long

Make catalog placing FTW win_a labels under win_b to let model learn from both time points. Make new combined catalog. 



In [18]:
ftw_catalog = catalog.query("dataset == 'ftw'")
ftw_catalog2 = ftw_catalog.copy()
ftw_catalog2["window_b"] = ftw_catalog2["window_a"]
ftw_catalog_long = (pd.concat([ftw_catalog, ftw_catalog2], axis=0)
                    .reset_index(drop=True))
ftw_catalog_long["window_a"] = ""
# .to_csv("../data/ftw-mappingafrica-combined-catalog.csv", index=False)

# Long FTW catalog
ftw_catalog_long.to_csv("../data/ftw-catalog-long.csv", index=False)

# Long combined
catalog_long = pd.concat([ftw_catalog_long, 
                          catalog.query("dataset == 'mappingafrica'")], axis=0)
# list(catalog_long.shape)
# catalog_long.head()
catalog_long.to_csv("../data/ftw-mappingafrica-combined-catalog-long.csv", 
                    index=False)

And new catalog that drops from validation set all presence-only data, but keeping Portugal this time in one (long2), and for consistency with previous dropping Portugal and presence only in the others (long3). 

In [ ]:
mask = ~(
    catalog_long['split'].isin(['validate', 'test'])
    & (catalog_long['null_prop'] > 0) 
)

catalogr = catalog_long[mask].copy().reset_index(drop=True)
print(len(catalogr), len(catalog_long))

(catalogr
 .to_csv("../data/ftw-mappingafrica-combined-catalog-long2.csv", index=False))

(catalogr.query("dataset == 'ftw'")
 .to_csv("../data/ftw-catalog-long2.csv", index=False))

mask = ~(
    catalog_long['split'].isin(['validate', 'test'])
    & (
        (catalog_long['null_prop'] > 0) | 
        (catalog_long['country'] == 'portugal')
    )
)

catalogr = catalog_long[mask].copy().reset_index(drop=True)
print(len(catalogr), len(catalog_long))

(catalogr
 .to_csv("../data/ftw-mappingafrica-combined-catalog-long3.csv", index=False))

(catalogr.query("dataset == 'ftw'")
 .to_csv("../data/ftw-catalog-long3.csv", index=False))

catalogr

FTW country list for validation metrics

In [47]:
ftw_catalog_long = pd.read_csv(
    "../data/ftw-catalog-long3.csv"
)
ftw_countries = (ftw_catalog_long.query("split == 'validate'")
 .country
 .value_counts()
 .reset_index()
 .sort_values("country")
 .drop("count", axis=1)
 )
with open("../scripts/ftw-countries.txt", "w") as f:
    for c in ftw_countries.country.tolist():
        f.write(f"{c}\n")


#### Mapping Africa and Full Catalogs with S2



In [37]:
s2_winb_catalog = pd.read_csv("../external/inputs/primary_merged_dedup.csv")
s2_winb_catalog.rename(columns={"chip_id": "name"}, inplace=True)

s2_dir = "/Users/LEstes/data/labels/cropland/mappingafrica-s2-256/window_b"

fns = [f for f in os.listdir(s2_dir)]
fns = pd.DataFrame(fns).rename(columns ={0: "window_b"})
fns["name"] = fns["window_b"].apply(lambda x: x.split("_")[0])

s2_winb_catalog = (
    pd.merge(s2_winb_catalog, fns, how="left", on="name")
    .rename(columns={"chip_id": "name", "s2_primary_date": "date"})
)
s2_winb_catalog = s2_winb_catalog[["name", "date", "window_b"]]
s2_winb_catalog["date"] = pd.to_datetime(
    s2_winb_catalog["date"], utc=True, errors="coerce"
).dt.strftime("%Y-%m-%d")
s2_winb_catalog

,name,date,window_b
0,AO0045466,2021-10-09,AO0045466_S2L2A_nomask.tif
1,AO0049230,2017-08-19,AO0049230_S2L2A_nomask.tif
2,AO0068052,2019-07-30,AO0068052_S2L2A_nomask.tif
3,AO0220548,2020-10-29,AO0220548_S2L2A_nomask.tif
4,AO0277614,2023-10-04,AO0277614_S2L2A_nomask.tif
...,...,...,...
39064,ZW1290563,2022-06-05,ZW1290563_S2L2A_nomask.tif
39065,ZW1323816,2022-06-05,ZW1323816_S2L2A_nomask.tif
39066,ZW1327144,2022-06-02,ZW1327144_S2L2A_nomask.tif
39067,ZW1330078,2018-08-02,ZW1330078_S2L2A_nomask.tif


In [42]:
fns

,window_b,name
0,GH0639365_S2L2A_nomask.tif,GH0639365
1,MZ0979620_S2L2A_nomask.tif,MZ0979620
2,ZM2491782_S2L2A_nomask.tif,ZM2491782
3,CG0737271_S2L2A_nomask.tif,CG0737271
4,NG0935668_S2L2A_nomask.tif,NG0935668
...,...,...
39011,NG1038617_S2L2A_nomask.tif,NG1038617
39012,TD4088262_S2L2A_nomask.tif,TD4088262
39013,CM0221406_S2L2A_nomask.tif,CM0221406
39014,TZ0933296_S2L2A_nomask.tif,TZ0933296


In [ ]:
ma_catalog = pd.read_csv("../data/mappingafrica-catalog.csv")
list(ma_catalog.shape)

# Experiments

## Functions

In [46]:
def write_yaml(template_path: str, output_path: str, updates: dict = None):
    """
    Write a YAML file from a template file, with optional updates.

    Args:
        template_path (str): Path to the base YAML template file.
        output_path (str): Path to the output YAML file.
        updates (dict, optional): Dictionary of keys/values to update.
    """

    def recursive_update(d, u):
        for k, v in u.items():
            if isinstance(v, dict) and isinstance(d.get(k), dict):
                recursive_update(d[k], v)
            else:
                d[k] = v

    with open(template_path, 'r') as f:
        config = yaml.safe_load(f)
        if updates:
            recursive_update(config, updates)

    class IndentDumper(yaml.SafeDumper):
        def increase_indent(self, flow=False, indentless=False):
            return super().increase_indent(flow, False)

    # custom representer for lists
    def represent_list(dumper, data):
        # flow style only if all elements are scalars
        if all(isinstance(x, (str, int, float, bool, type(None))) for x in data):
            return dumper.represent_sequence("tag:yaml.org,2002:seq", data, 
                                             flow_style=True)
        else:
            return dumper.represent_sequence("tag:yaml.org,2002:seq", data, 
                                             flow_style=False)

    IndentDumper.add_representer(list, represent_list)

    with open(output_path, 'w') as f:
        yaml.dump(
            config,
            f,
            Dumper=IndentDumper,
            default_flow_style=False,  # keep dicts block-style
            sort_keys=False,
            indent=2,
            allow_unicode=True
        )


## Naming conventions

Model config files are general named according to the following recipe:

- `<model>_<backbone>-<loss>-<norm>-<augmentation>-<epochs><-patience>-<catalog>`

Although this recipe has varied a bit as experiments have evolved, primarily in the early phase where different settings were being evaluated, and as efforts to shorten names were applied (the config is applied to model outputs folders and metrics files, which become cumbersome).

The pattern above is generally for models trained on just the FTW dataset, where there were different permutations of the single time point catalog initially (window_b only and window_b + window_a combined under window_b [long catalogs]).  

For other catalogs--full and Mapping Africa only--the recipe is as follows:

- `<full|ma>-<model>_<backbone>-<loss>-<norm>-<augmentation>-<epochs><-patience>-<catalog>`

In all cases, there are variations within most components also. 

- For FTW baseline architecture models (U-Net with EfficientNet-b3), `<model>_<backbone>` is either simply `ftwbaseline` or `ftwbase`
- For the model approximating our standard Mapping Africa architecture (U-Net with VGG19), `<model>_<backbone>` is either `ma-approximate` or `ma-approx` or `ma-approxbase`
- For other variants, the full model and backbone names are used, e.g., `unet_cvnxt_sm` or `unet_cvnxt_sm` for U-Net with ConvNeXt Small backbone pretrained on ImageNet-22K. 
- Losses vary, but are fairly self-explanatory, although locally-weighted tversky focal is often not mentioned (in ma-approx models), where it is standard, or called `localtversky` or `loctver`, and finally `ltfl`.  Similarly weighted cross-entropy is left unmentioned for FTW baselines models. The loss in these cases is only made explicit when different. 
- The general augmentation are abbreviated out separately, specifying additional augmentations added on top of what FTW used by default (rotation, v and h flips, sharpness). These include `rescale` or `rscl` and `rscl2`, which is rescaling with different parameters, and `brightness` or `brgt`. 
    - We have settled now onto a standard package that adds brightness and rescaling with FTW settings, so names containg `sta` now indicate the final standard augmentation package. 
- Epochs are not listed unless longer than 100. Similarly with patience. As both of these should be matched, the standard now for trains longer than 100 is e.g. `ep300` or epochs of 300 with matching patience.
- `<catalog>` is not always provided either. Older models runs did not mention this, as they only used window_b. Newer FTW or full catalog runs end typically in `longcat` to show that they use a single window, but the model is shown both window A and B. Later experiments drop this again because all single time point runs involving the FTW catalog now use longcat. 






## Single time point

### FTW catalog

#### FTW baseline 
All experiments here are FTW baseline model, window B only

##### Setup

Below we set up a yaml for each experiment. Provide the following:

- `cfg_name`: name of the config/experiment file (without .yaml)
- `update`: dictionary of changes to make to the base config

Also define a global `home_dir` for the path to the repo containing the catalog. That's done once in the first cell. 

In [8]:
home_dir = "~/projects"
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-catalog2.csv",
        )
    )
)

##### # 1 Default

With updated optimal weights added

In [ ]:
cfg_name = "ftwbaseline-exp1"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 2 Locally-weighted tversky focal loss


In [93]:
cfg_name = "ftwbaseline-localtversky-exp2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 3 Local min-max normalization, across bands


In [94]:
cfg_name = "ftwbaseline-minmax_lab-exp3"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="lab",
    global_stats=None,
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 3a Local min-max normalization, across bands, 1% clip

In [19]:
cfg_name = "ftwbaseline-minmax_lab-exp3a"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="lab",
    img_clip_val=1,
    global_stats=None,
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 4 Global min-max normalization, across bands

Using the 1st and 99th percentiles as min-max. 


In [18]:
cfg_name = "ftwbaseline-minmax_gab-exp4"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

# Ensure global_stats exists before popping
global_stats = update["data"]["init_args"].get("global_stats", {})
global_stats.pop("mean", None)
global_stats.pop("std", None)
update["data"]["init_args"]["global_stats"] = global_stats

update["data"]["init_args"].update(dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### # 4a Per band Z-value, global stats

In [20]:
cfg_name = "ftwbaseline-zvalue_gpb-exp4a"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    normalization_stat_procedure="gpb",
    global_stats={"mean": [874.538, 876.9152, 641.7087, 2925.554], 
                  "std": [759.7574, 648.3951, 619.3338, 1083.3489]},
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 5 - photometric augmentations

In [95]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ftwbaseline-photometric-exp5"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    aug_list=augs + ["brightness", "contrast", "gaussian_noise"]
)    
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 6 - satslidemix

In [96]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ftwbaseline-satslide-exp6"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    aug_list=augs + ["satslidemix"]
)    
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 7 - rescale

In [97]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ftwbaseline-rescale-exp7"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    aug_list=augs + ["rescale"]
)    
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 8 - Tversky and min-max GAB

In [13]:
cfg_name = "ftwbaseline-localtversky-minmax_gab-exp8"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### MA Approximate baseline model

In [10]:
cfg_name = "ftw-ma-approximate-exp1"
# update = dict(
#     data=dict(
#         init_args=dict(
#             catalog=f"{home_dir}/"\
#                 "ftw-mappingafrica-integration/data/ftw-catalog2.csv"
#         )
#     )
# )
# update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
# write_yaml("../configs/ma-approximate-baseline.yaml", 
#            f"../configs/{cfg_name}.yaml", 
#            updates=update)

augs = ["rotation", "hflip", "vflip", "sharpness"]
# cfg_name = "ma-approximate-baseline"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "contrast", "gaussian_noise", "rescale", "gamma"],
        normalization_strategy="min_max",
        normalization_stat_procedure="lab",
        global_stats=None,
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

### Mapping Africa catalog


In [43]:
home_dir = "~/projects"
augs = ["rotation", "hflip", "vflip", "sharpness"]
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/mappingafrica-catalog.csv"
        )
    )
)

#### MA Approximate baseline 

We will train a model close to the variant we used on Planet imagery for mapping countries in Africa, initially trained on just the Mapping Africa labels.

##### Near default

These settings are about as close as possible to those originally used

In [12]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ma-approximate-baseline"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "contrast", "gaussian_noise", "rescale", "gamma"],
        normalization_strategy="min_max",
        normalization_stat_procedure="lab",
        global_stats=None,
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### MA approximate with only brightness and rescale

Brightness now has 50% probability, and rescale parameters are now different tracking FTW's. This steps it down a bit now. 

In [15]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ma-approxbase-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="lab",
        global_stats=None,
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### MA approximate, minmax GAB, brightness and rescale

Using MA global stats, between 1st and 99th percentiles: 

- min: 203.544
- max: 4590.354

In [8]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ma-approxbase-mnmx_gab-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### MA approximate, SMP tversky, minmax GAB, brightness and rescale


In [15]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ma-approx-smptver-mnmx_gab-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### MA approx, logcoshdice, minmax GAB, brightness and rescale

In [ ]:
cfg_name = "ma-approx-logcoshdice-mnmx_gab-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="logcoshdice", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### FTW baseline 

##### # 1 Standard

In [ ]:
cfg_name = "ma-ftwbaseline-exp1"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"] = dict(
    catalog=f"{home_dir}/"\
        f"ftw-mappingafrica-integration/data/mappingafrica-catalog.csv"   
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 2 Min-max, lab

In [ ]:
cfg_name = "ma-ftwbaseline-exp2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"].update(dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="lab",
    global_stats=None,
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline with local tversky

Let's put this more on an equal footing with the MA approximate baseline with local tversky loss and similar augmentations. 



In [19]:
cfg_name = "ma-ftwbase-localtversky-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="lab",
        global_stats=None,
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, local tversky, minmax GAB, brightness and rescale

In [5]:
cfg_name = "ma-ftwbase-loctversky-mnmx_gab-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, logcoshdice, minmax GAB, brightness and rescale

In [7]:
cfg_name = "ma-ftwbase-logcoshdice-mnmx_gab-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="logcoshdice", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, SMP tversky, minmax GAB, brightness and rescale

In [ ]:
cfg_name = "ma-ftwbase-smptversky-mnmx_gab-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, tversky focal, standard augmentations

In [9]:
cfg_name = "ma-ftwbase-tfl-mmgab-sta"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tverskyfocal", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, tversky focal CE, standard augmentations

In [47]:
cfg_name = "ma-ftwbase-tflce-mmgab-sta"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        loss="tverskyfocalce", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, local tversky focal CE, standard augmentations

In [12]:
cfg_name = "ma-ftwbase-localtflce-mmgab-sta"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tverskyfocalce", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### FTW baseline, local tversky focal, standard augmentations

This one is a redo to check that new version gives the same results as before.

In [13]:
cfg_name = "ma-ftwbase-localtfl-mmgab-sta"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### ConvNeXt, local tversky, minmax GAB, brightness and rescale

In [30]:
cfg_name = "ma-unet_cvnxt_sm-ltfl-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=100,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


#### Efficientnet-b7, local tversky, minmax GAB, brightness and rescale

In [32]:
ep = 100
cfg_name = "ma-unet-effb7-ltfl-mnmx_gab-sta-ep100"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=ep,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="efficientnet-b7",
        patience=ep,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [203.544], "max": [4590.354], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

### Full catalog

Setting up runs that will pull from the full catalog, starting with the FTW baseline model.

We'll also specify some validation sets to evaluate results. To start we will use the separated FTW and Mapping Africa "global" validation samples, without confining them to specific countries. 

In [ ]:
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-mappingafrica-combined-catalog2.csv"   
        )
    )
)


#### FTW Baseline


##### # 1 Standard settings

In [ ]:
cfg_name = "fullcat-ftwbaseline-exp1"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 2 LAB

In [120]:
cfg_name = "fullcat-ftwbaseline-exp2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["data"]["init_args"].update(dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="lab",
    global_stats=None,
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### # 3 Locally-weighted tversky focal loss



In [14]:
cfg_name = "fullcat-ftwbaseline-exp3"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### MA approximate baseline

In [ ]:
cfg_name = "fullcat-ma-approximate-baseline-exp1"
update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-mappingafrica-combined-catalog2.csv"   
        )
    )
)
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
write_yaml("../configs/ma-approximate-baseline.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

## Single time point, long catalog

Re-running training for the full catalog and FTW only versions using the new long catalog.



### FTW catalog

In [33]:
home_dir = "~/projects"
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-catalog-long2.csv",
        )
    )
)

#### FTW baseline

##### Locally-weighted tversky focal loss, redo

In [16]:
cfg_name = "ftwbaseline-localtversky-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### Tversky and min-max GAB, redo

In [21]:
cfg_name = "ftwbaseline-localtversky-minmax_gab-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(dict(
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### Tversky, min-max GAB, photometric augmentations

In [26]:
augs = ["rotation", "hflip", "vflip", "sharpness"]
cfg_name = "ftwbaseline-localtversky-minmax_gab-photometric-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["brightness", "contrast", "gaussian_noise", "gamma"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### Tversky, min-max GAB, photometric augmentations, rescale 

In [28]:
cfg_name = "ftwbaseline-localtversky-minmax_gab-photometric-rescale-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["rescale", "brightness", "contrast", "gaussian_noise", 
                     "gamma"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### Tversky, min-max GAB, brightness only, rescale

Back to ftw-baseline. Note I tweaked the augmentation code to make this align now with FTW's brightness augmentation approach. 

In [32]:
cfg_name = "ftwbaseline-localtversky-minmax_gab-brightness-rescale-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="localtversky"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["rescale", "brightness"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### SMP Tversky, min-max GAB, brightness only, rescale

In [6]:
cfg_name = "ftwbase-smptversky-mnmx_gab-bright-rescale-longcat"
augs = ["rotation", "hflip", "vflip", "sharpness"]
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="tversky"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["rescale", "brightness"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### Dice, min-max GAB, brightness only, rescale

In [9]:
cfg_name = "ftwbase-dice-mnmx_gab-bright-rescale-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="dice"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["rescale", "brightness"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

##### logCoshDice, min-max GAB, brightness only, rescale

In [10]:
cfg_name = "ftwbase-logcoshdice-mnmx_gab-bright-rescale-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(init_args=dict(loss="logcoshdice"))
update["data"]["init_args"].update(dict(
    aug_list=augs + ["rescale", "brightness"],
    normalization_strategy="min_max",
    normalization_stat_procedure="gab",
    global_stats={"min": [68.438525], "max": [5772.288821], 
                  "mean": None, "std": None},
))
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### ConvNeXt backbone

In [20]:
home_dir = "~/projects"
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                "ftw-mappingafrica-integration/data/ftw-catalog-long2.csv",
        )
    )
)

##### Full augs, local-tversky

We'll try out ConvNeXt small, pre-trained on ImageNet-22k. **Note**: the yaml should have been named differently (convnext_small-...). Also note this one had the wrong normalization stats procedure set (should have been gab, was lab)

In [ ]:
cfg_name = "ftwbaseline-tversky-allaugs-convnext_small_in22k-longcat"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + \
            ["rescale", "brightness", "contrast", "gaussian_noise", "gamma"],
        normalization_strategy="min_max",
        normalization_stat_procedure="lab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### ConvNext, local tversky, brightness & rescale, 300 epochs

Ditching the additional photometrics augmentations since those are probably disappearing, and so the main thing here is to see if the model keeps learning. Validation curves from the model above suggested it had substantial improvements left. GAB normalization added as before. 

Note: I manually updated this one to 400 epochs after seeing continued gains at 300 epochs, and then again to 500 epochs.

In [23]:
cfg_name = "unet_cvnxt_sm_in22k-loctversky-brgt-rescl-e300-longcat"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=300,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### ConvNext, local tversky, brightness & rescale, 300 epochs, different patience

The ups and downs in learning on the long run from the one above seem to be resetting learning rate due to patience of 100 and cosine annealing. Will match patience to epochs to make continual progress. 

In [16]:
cfg_name = "unet_cvnxt_sm-loctver-brgt-rescl-e300-p300-longcat"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=300,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
        patience=300,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### ConvNext, local tversky, brightness & rescale, 400 epochs

The ups and downs in learning on the long run from the one above seem to be resetting learning rate due to patience of 100 and cosine annealing. Will match patience to epochs to make continual progress. 

In [22]:
cfg_name = "unet_cvnxt_sm-ltfl-sta-ep400-longcat"
ep = 400
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=ep,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
        patience=ep,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### ConvNext, logcoshdice, brightness & rescale, 400 epochs

Trying best performing logcoshdice, which seems better than local tversky, although weak on object recall. 

In [13]:
cfg_name = "unet_cvnxt_sm_in22k-logcoshdice-brgt-rescl-e400-longcat"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=400,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="logcoshdice", 
        backbone="tu-convnext_small.fb_in22k",
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


##### ConvNext, logcoshdice, brightness & rescale, 300 epochs, 300 patience

Above with LR not resetting due to patience being too short.

In [17]:
cfg_name = "unet_cvnxt_sm-logcoshdice-brgt-rescl-e400-p300-longcat"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=400,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="logcoshdice", 
        backbone="tu-convnext_small.fb_in22k",
        patience=300,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


#### EfficientNet-b7, 400 epochs

In [34]:
ep = 400
cfg_name = "unet-effb7-ltfl-mmgab-sta-ep400-longcat"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=ep,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="efficientnet-b7",
        patience=ep,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [68.438525], "max": [5772.288821], 
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

### Full catalog

In [6]:
home_dir = "~/projects"
augs = ["rotation", "hflip", "vflip", "sharpness"]
base_update = dict(
    data=dict(
        init_args=dict(
            catalog=f"{home_dir}/"\
                f"ftw-mappingafrica-integration/data/"\
                    "ftw-mappingafrica-combined-catalog-long2.csv"   
        )
    )
)


#### FTW base, local tversky, GAB, brightness & rescale, 100 epochs

Maybe extend longer depending on curves. Keeping patience at 100. 

In [23]:
cfg_name = "full-ftwbase-loctver-mnmx_gab-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### FTW base, SMP tversky, GAB, brightness & rescale, 100 epochs



In [19]:
cfg_name = "full-ftwbase-smptver-mnmx_gab-brgt-rescl2"
update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")

update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tversky", 
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3], 
                      "mean": None, "std": None},
    ) 
)

write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### MA approximate, local tversky, GAB, brightness & rescale, 100 epochs


In [25]:
cfg_name = "full-ma-approx-mnmx_gab-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### MA approximate, smp tversky, GAB, brightness & rescale, 100 epochs


In [17]:
cfg_name = "full-ma-approx-smptver-mnmx_gab-brgt-rescl2"

update = base_update.copy()
update["trainer"] = dict(default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="tversky", 
        backbone="tu-vgg19_bn.tv_in1k",
        patch_weights=True,
        model_kwargs=dict(drop_rate=0.1)
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["brightness", "rescale"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3], 
                      "mean": None, "std": None},
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)

#### ConvNext, local tversky, brightness & rescale, 300 epochs, 300 patience


In [27]:
ep = 400
cfg_name = "full-unet_cvnxt_sm-ltfl-sta-ep400"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=ep,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="tu-convnext_small.fb_in22k",
        patience=ep,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3],
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)


#### EfficientNet-b7, 400 epochs

In [7]:
ep = 400
cfg_name = "full-unet-effb7-ltfl-mmgab-sta-ep400"
update = base_update.copy()
update["trainer"] = dict(
    max_epochs=ep,
    default_root_dir=f"~/working/models/{cfg_name}")
update["model"] = dict(
    init_args=dict(
        class_weights=None,
        loss="localtversky", 
        backbone="efficientnet-b7",
        patience=ep,
    )
)
update["data"]["init_args"].update(
    dict(
        aug_list=augs + ["rescale", "brightness"],
        normalization_strategy="min_max",
        normalization_stat_procedure="gab",
        global_stats={"min": [89.1], "max": [5528.3],
                      "mean": None, "std": None}
    ) 
)
write_yaml("../configs/template-hpc-config.yaml", 
           f"../configs/{cfg_name}.yaml", 
           updates=update)